# Imports
    

In [1]:
#First we'll start with importing the libraries that we need to perform 
#our cleaning, prep, analysis other steps will proceed

In [2]:
import subprocess    
import acquire
import prepare

import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
#import acquire.py
#import prepare.py

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
def get_telco_data():
    '''
    This function reads in telco data from Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('telco.csv'):
        
        # If csv file exists read in data from csv file.
        df = pd.read_csv('telco.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame
        df = new_telco_data()
        
        # Cache data
        df.to_csv('telco.csv')
        
    return df

# Acquire

Acquire:
The data came from IBM initially, as a randomly generated dataset so that others could have a set to work on and learn form. Last information about the data set was provided on Thu July 11, 2019 08:15 AM. 

https://community.ibm.com/community/user/businessanalytics/blogs/steven-macko/2019/07/11/telco-customer-churn-1113

I accessed the data via pydataset and imported via seaborn's load dataset function



In [4]:
#here we load the dataset and set it to df so it can be referenced later


In [5]:
def get_db_url(db):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

'''
user = 'mirzakhani_1937'
password = 'p4h6GGRgeslnWcjIFZoViIY3b92EaC18'
host = 'data.codeup.com'
'''

In [6]:
def new_telco_data():
    '''
    This function reads the telco data from the Codeup db into a df.
    '''
    sql_query = """
                select * from customers
                join contract_types using (contract_type_id)
                join internet_service_types using (internet_service_type_id)
                join payment_types using (payment_type_id)
                """
    
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_db_url('telco_churn'))
    
    return df



In [7]:
def get_telco_data():
    '''
    This function reads in telco data from Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('telco.csv'):
        
        # If csv file exists read in data from csv file.
        df = pd.read_csv('telco.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame
        df = new_telco_data()
        
        # Cache data
        df.to_csv('telco.csv')
        
    return df

In [8]:
df = new_telco_data() #here's where it actually happens

In [9]:
df #checking to make sure everything loaded 

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,2,0002-ORFBO,Female,0,Yes,Yes,9,Yes,...,Yes,Yes,No,Yes,65.60,593.3,No,One year,DSL,Mailed check
1,2,1,1,0003-MKNFE,Male,0,No,No,9,Yes,...,No,No,Yes,No,59.90,542.4,No,Month-to-month,DSL,Mailed check
2,1,2,1,0004-TLHLJ,Male,0,No,No,4,Yes,...,No,No,No,Yes,73.90,280.85,Yes,Month-to-month,Fiber optic,Electronic check
3,1,2,1,0011-IGKFF,Male,1,Yes,No,13,Yes,...,No,Yes,Yes,Yes,98.00,1237.85,Yes,Month-to-month,Fiber optic,Electronic check
4,2,2,1,0013-EXCHZ,Female,1,Yes,No,3,Yes,...,Yes,Yes,No,Yes,83.90,267.4,Yes,Month-to-month,Fiber optic,Mailed check
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2,1,2,9987-LUTYD,Female,0,No,No,13,Yes,...,Yes,No,No,No,55.15,742.9,No,One year,DSL,Mailed check
7039,1,2,1,9992-RRAMN,Male,0,Yes,No,22,Yes,...,No,No,Yes,Yes,85.10,1873.7,Yes,Month-to-month,Fiber optic,Electronic check
7040,2,1,1,9992-UJOEL,Male,0,No,No,2,Yes,...,No,No,No,Yes,50.30,92.75,No,Month-to-month,DSL,Mailed check
7041,2,1,3,9993-LHIEB,Male,0,Yes,Yes,67,Yes,...,Yes,No,Yes,No,67.85,4627.65,No,Two year,DSL,Mailed check


Data was downloadeded from Codeup db, 25th Oct
Size = 891 rows and 15 columns
observations represent the different employees at telco 
Columns represent the different characteristics that were recorded at the original time of creation

# Prepare 

-To prepare the data, we remove columns that are duplicated such as payment type id, internet service type is, conract type id, and customer id.

-We also remove any null values stored as white space

-Additionally the 'total charges" column gets converetd to a float, and binary categories get changed to numeric values, gender, partners, dependents, phone service, and churn status.

-multiple lines, online security, online backup, device proteciton,tech support, streaming tv, streaming movies, contact type, internet service type, payment type -> these are completed/filled out with dummy variables so there are no rows with blank cells

In [10]:
#this creates a function that splits the data into three groups
def split_telco_data(df):
    '''
    This function performs split on telco data, stratify churn.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df.churn)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123, 
                                   stratify=train_validate.churn)
    return train, validate, test




In [11]:
df.gender


0       Female
1         Male
2         Male
3         Male
4       Female
         ...  
7038    Female
7039      Male
7040      Male
7041      Male
7042      Male
Name: gender, Length: 7043, dtype: object

In [12]:
def prep_telco_data(df):
    # Drop duplicate columns
    df.drop(columns=['payment_type_id', 'internet_service_type_id', 'contract_type_id', 'customer_id'], inplace=True)
       
    # Drop null values stored as whitespace    
    df['total_charges'] = df['total_charges'].str.strip()
    df = df[df.total_charges != '']
    
    # Convert to correct datatype
    df['total_charges'] = df.total_charges.astype(float)
    
    # Convert binary categorical variables to numeric
    df['gender_encoded'] = df.gender.map({'Female': 1, 'Male': 0})
    df['partner_encoded'] = df.partner.map({'Yes': 1, 'No': 0})
    df['dependents_encoded'] = df.dependents.map({'Yes': 1, 'No': 0})
    df['phone_service_encoded'] = df.phone_service.map({'Yes': 1, 'No': 0})
    df['paperless_billing_encoded'] = df.paperless_billing.map({'Yes': 1, 'No': 0})
    df['churn_encoded'] = df.churn.map({'Yes': 1, 'No': 0})
    
    df['gender_encoded'].replace(['Female', 'Male'],[1,0], inplace=True)
    df['contract_type'].replace(['Two year', 'One year', 'Month-to-month'],[2,1,0], inplace=True)
    
    # Get dummies for non-binary categorical variables
    dummy_df = pd.get_dummies(df[['multiple_lines', \
                              'online_security', \
                              'online_backup', \
                              'device_protection', \
                              'tech_support', \
                              'streaming_tv', \
                              'streaming_movies', \
                              'contract_type', \
                              'internet_service_type', \
                              'payment_type']], dummy_na=False, \
                              drop_first=True)
    
    # Concatenate dummy dataframe to original 
    df = pd.concat([df, dummy_df], axis=1)
    
    # split the data
   # train, validate, test = split_telco_data(df)
    
    #return train, validate, test

In [13]:
 df['gender'].replace(['Female', 'Male'],[1,0], inplace=True)

In [14]:
df['contract_type'].replace(['Two year', 'One year', 'Month-to-month'],[2,1,0], inplace=True)

In [15]:
df['partner'].replace(['Yes', 'No'],[1,0], inplace=True)

In [16]:
df['dependents'].replace(['Yes', 'No'],[1,0], inplace=True)

In [17]:
df['contract_type'].replace(['Two year', 'One year', 'Month-to-month'],[2,1,0], inplace=True)

In [18]:
df['phone_service'].replace(['Yes', 'No'],[1,0], inplace=True)

In [19]:
df['paperless_billing'].replace(['Yes', 'No'],[1,0], inplace=True)

In [20]:
df['churn'].replace(['Yes', 'No'],[1,0], inplace=True)

In [21]:
df['multiple_lines'].replace(['Yes', 'No'],[1,0], inplace=True)

In [22]:
df['online_security'].replace(['Yes', 'No'],[1,0], inplace=True)

In [23]:
df['online_backup'].replace(['Yes', 'No'],[1,0], inplace=True)	

In [31]:

df['device_protection'].replace(['Yes', 'No'],[1,0], inplace=True)	
df['tech_support'].replace(['Yes', 'No'],[1,0], inplace=True)	
df['streaming_tv'].replace(['Yes', 'No'],[1,0], inplace=True)	
df['streaming_movies'].replace(['Yes', 'No'],[1,0], inplace=True)	
df['paperless_billing'].replace(['Yes', 'No'],[1,0], inplace=True)	
df['monthly_charges'].replace(['Yes', 'No'],[1,0], inplace=True)	
df['total_charges'].replace(['Yes', 'No'],[1,0], inplace=True);
#df['internet_service_type'].replace(['DSL' 'Fiber optic'],[1,0], inplace=True)	
df['dependents'].replace(['Yes', 'No'],[1,0], inplace=True)	
#df['payment_type'].replace(['Electronic check', 'Mailed check'],[1,0], inplace=True)	



df['tech_support'].replace(['Yes', 'No'],[1,0], inplace=True)	
df['streaming_tv'].replace(['Yes', 'No'],[1,0], inplace=True)	
df['streaming_movies'].replace(['Yes', 'No'],[1,0], inplace=True)	
df['paperless_billing'].replace(['Yes', 'No'],[1,0], inplace=True)	
df['total_charges'].replace(['Yes', 'No'],[1,0], inplace=True)	
df['churn'].replace(['Yes', 'No'],[1,0], inplace=True)	
df['contract_type'].replace(['Yes', 'No'],[1,0], inplace=True)	
df['internet_service_type'].replace(['Yes', 'No'],[1,0], inplace=True)	
df['payment_type'].replace(['Yes', 'No'],[1,0], inplace=True)	






tech_support	streaming_tv	streaming_movies	paperless_billing
monthly_charges	total_charges	churn	contract_type	internet_service_type	payment_type



SyntaxError: invalid syntax (681203499.py, line 29)

In [25]:
df

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,2,0002-ORFBO,1,0,1,1,9,1,...,Yes,Yes,No,1,65.60,593.3,0,1,DSL,Mailed check
1,2,1,1,0003-MKNFE,0,0,0,0,9,1,...,No,No,Yes,0,59.90,542.4,0,0,DSL,Mailed check
2,1,2,1,0004-TLHLJ,0,0,0,0,4,1,...,No,No,No,1,73.90,280.85,1,0,Fiber optic,Electronic check
3,1,2,1,0011-IGKFF,0,1,1,0,13,1,...,No,Yes,Yes,1,98.00,1237.85,1,0,Fiber optic,Electronic check
4,2,2,1,0013-EXCHZ,1,1,1,0,3,1,...,Yes,Yes,No,1,83.90,267.4,1,0,Fiber optic,Mailed check
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2,1,2,9987-LUTYD,1,0,0,0,13,1,...,Yes,No,No,0,55.15,742.9,0,1,DSL,Mailed check
7039,1,2,1,9992-RRAMN,0,0,1,0,22,1,...,No,No,Yes,1,85.10,1873.7,1,0,Fiber optic,Electronic check
7040,2,1,1,9992-UJOEL,0,0,0,0,2,1,...,No,No,No,1,50.30,92.75,0,0,DSL,Mailed check
7041,2,1,3,9993-LHIEB,0,0,1,1,67,1,...,Yes,No,Yes,0,67.85,4627.65,0,2,DSL,Mailed check


In [ ]:
#df = new_telco_data()

In [ ]:
prep_telco_data(df)

In [ ]:
#df.gender
#df.contract_type

In [ ]:

#this creates a function that splits the data into three groups
def split_telco_data(df):
    '''
    This function performs split on telco data, stratify churn.
    Returns train, validate, and test dfs.
    '''
    train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df.churn)
    train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123, 
                                   stratify=train_validate.churn)
    return train, validate, test




In [ ]:
train, validate, test = split_telco_data(df)

In [ ]:
train

In [ ]:
# split into train, validate, test
#train, validate, test = train_validate_test_split(df, target='species', seed=123)

# create X & y version of train, where y is a series with just the target variable and X are all the features. 

x_train = train.drop(columns=['churn'])
y_train = train.churn

x_validate = validate.drop(columns=['churn'])
y_validate = validate.churn

x_test = test.drop(columns=['churn'])
y_test = test.churn

In [ ]:
x_train

In [ ]:
#attempting to add analysis

In [ ]:
rf = RandomForestClassifier(max_depth=3, random_state=123)

In [ ]:
rf.fit(x_train, y_train)

# Explore


# Questions:
###  What contract type has the largest number of customers?
###  Does payment type have any kind of correlation on a customers ability to keep making their bill payments on time?
### Do customers with more services maintain their accounts for longer periods of time?
### Does being partnered with someone correlate with any difference in services?




In [ ]:
train.info

In [ ]:
train.internet_service_type.value_counts() #Just under 1/4 of CS no Internet

In [ ]:
train.phone_service.value_counts() #most cs have phone service

In [ ]:
train.tenure.value_counts() # Interesting point: highest and lowest num of months in 
#the first two places for highest occurence, followed by the 2nd,3rd and 2nd to last 
# num of months. first 5 values are high/low ends of the spectrum. Does 0 mean new CS?

In [ ]:
train.contract_type.value_counts() #surprising number of month to month
#previous visualizations show that many MtM customers churn compared to 
#longer contract types

In [ ]:
train.payment_type.value_counts() #weirdly even split between non E-check

In [ ]:
train.churn.value_counts()

In [ ]:
sns.relplot(data=train, x="contract_type", y='total_charges')